### Setup Libraries and Splinter Browser Configuration

In [1]:
# Import dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import requests
import time
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 101.0.4951
[WDM] - Get LATEST chromedriver version for 101.0.4951 google-chrome
[WDM] - Driver [C:\Users\austr\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


### NASA Mars News Scrape
* Collect latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [3]:
# Visit Mars News URL
MarsNews_URL = 'http://redplanetscience.com'
browser.visit(MarsNews_URL)
time.sleep(1)

# Scrape into BeautifulSoup by creating HTML object and parser HTML
html = browser.html
soup = bs(html, "html.parser")

# Find and save News Title under <div> tag with class='content_title'
NewsTitle = soup.find('div', class_='content_title').text

# Print News Title
print(NewsTitle)

# Find and save Praragraph text under <div> tag with class='article_teaser_body'
NewsParagraph = soup.find('div', class_='article_teaser_body').text

# Print Paragraph text
print(NewsParagraph)

NASA, ULA Launch Mars 2020 Perseverance Rover Mission to Red Planet
The agency's Mars 2020 mission is on its way. It will land at Jezero Crater in about seven months, on Feb. 18, 2021. 


### JPL Mars Space Images (Featured Images) Scrape
* Visit the url for the Featured Space Image page here.
* Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.
* Make sure to find the image url to the full size .jpg image.
* Make sure to save a complete url string for this image.

In [4]:
# Visit JPL Featured Space Image URL
JPLimages_URL = 'https://spaceimages-mars.com/'
browser.visit(JPLimages_URL)
time.sleep(1)

# Scrape into BeautifulSoup by creating HTML object and parser HTML
html = browser.html
soup = bs(html, "html.parser")


In [5]:
# Get image from URL
img = [i.get("src") for i in soup.find_all("img", class_="headerimage fade-in")]
img[0]

'image/featured/mars2.jpg'

In [6]:
# Save URL string for image
featured_image_url = JPLimages_URL + img[0]
featured_image_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

In [7]:
# Stop browser
browser.quit()

### Mars Facts Scrape
* Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
* Use Pandas to convert the data to a HTML table string.

In [22]:
# Visit Mars Facts URL
MarsFacts_URL = 'https://galaxyfacts-mars.com/'

# Scrape table from URL using pandas
FactsTable = pd.read_html(MarsFacts_URL)

# Create table dataframe
df=FactsTable[1]
df.columns=['Discription', 'Value']
df

,Discription,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 ( Phobos & Deimos )
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [23]:
# Save table to html
FactsHTML = df.to_html()

# Print table
print(FactsHTML)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Discription</th>
      <th>Value</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 ( Phobos &amp; Deimos )</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</th>
  

### Mars Hemispheres
* Visit the Astrogeology site here to obtain high resolution images for each of Mar's hemispheres.
* You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.
* Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys img_url and title.
* Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

In [26]:
# Visit Mars Hemisphere URL
MarsHem_URL = 'https://marshemispheres.com/'
URL_path = requests.get(MarsHem_URL)

In [27]:
# Define Path with BeautifulSoup
soup = bs(URL_path.content)

In [28]:
# Find <div> with class_="item"
div_item = soup.find_all("div", class_="item")

In [33]:
# Loop through Links
Links = []
for i in div_item:
    for a in i.find_all('a', href=True):
        if a.text:
            Links.append(a['href'])

In [30]:
# Display Links
FullLinks = [MarsHem_URL + i for i in Links]
FullLinks

['https://marshemispheres.com/cerberus.html',
 'https://marshemispheres.com/schiaparelli.html',
 'https://marshemispheres.com/syrtis.html',
 'https://marshemispheres.com/valles.html']

In [31]:
# Extract image links
def extract_image(MarsHem_URL):
    respons = requests.get(MarsHem_URL)
    soup = bs(respons.text, "html.parser")
    image = soup.find_all("img", class_="wide-image")
    img = [i.get("src") for i in image]
    return "https://marshemispheres.com/" + img[0]

In [32]:
# Display extartced image links
ImgURL = [extract_image(i) for i in FullLinks]
ImgURL

['https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg',
 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg',
 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg',
 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg']

In [34]:
# Define image titles
def img_title(MarsHem_URL):
    response = requests.get(MarsHem_URL)
    soup = bs(response.text, "html.parser")
    title = soup.find_all("h2", class_="title")
    title_text = [i.text for i in title]
    return title_text[0]

In [35]:
# Display image titles
title = [img_title(i) for i in FullLinks]
title

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [36]:
# Create image title links
list_tups = list(zip(title, ImgURL))
list_tups

[('Cerberus Hemisphere Enhanced',
  'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'),
 ('Schiaparelli Hemisphere Enhanced',
  'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'),
 ('Syrtis Major Hemisphere Enhanced',
  'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'),
 ('Valles Marineris Hemisphere Enhanced',
  'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg')]

In [37]:
# Create dictionay of images and image titles
def get_dict_list(keys, list_of_tups):
    list_of_dict = [dict(zip(keys, values)) for values in list_of_tups]
    return list_of_dict

In [38]:
# Set keys
keys = ("title", "ImgURL")
hemisphere_img_URLs = get_dict_list(keys, list_tups)

In [39]:
# Display Mars hemisphere image and title URLS
hemisphere_img_URLs

[{'title': 'Cerberus Hemisphere Enhanced',
  'ImgURL': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'ImgURL': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'ImgURL': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'ImgURL': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]